In [33]:
#experiments_dev = ["WFV1_unet", "WF1_unet_all", "WF1_unet_bgr", "WF1_unet_rgbiswir", "WF1_unet_rgbi",
#                  "WF1_hrnet_rgbbs32", "WF1_hrnet_allbs32", "WF2_unet", "WF2_hrnetsmall_rgb","WFV1_scnn20", "WF1_scnn_bgr",
#                   "WF1_simplecnn_rgbi", "WF1_simplecnn_rgbiswir", "WF2_unet_extra","WF2_Unet_unused_from_checkpoint"]

experiments_dev = ["WFV1_unet", "WF1_unet_all", "WF1_unet_rgbiswir", "WF1_unet_rgbi",
            "WF1_hrnet_rgbbs32",'WF1_unet_all',"WF2_unet","WF2_hrnetsmall_rgb", "WF2_Unet_unused_from_checkpoint"]
#,"MNDWI" ]


In [23]:
from ml4floods.data import utils
from ml4floods.models.utils import metrics
import pandas as pd
from ml4floods.models.config_setup import load_json

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"X:\home\kike\json_creds\ml4cc-general-access_request_pays.json"

In [24]:
def get_metrics_percode(mets):
    iou_per_code = pd.DataFrame(metrics.group_confusion(mets["confusions"],mets["cems_code"], metrics.calculate_iou,
                                                        label_names=[f"IoU_{l}"for l in ["land", "water", "cloud"]]))

    recall_per_code = pd.DataFrame(metrics.group_confusion(mets["confusions"],mets["cems_code"], 
                                                           metrics.calculate_recall,
                                                           label_names=[f"Recall_{l}"for l in ["land", "water", "cloud"]]))

    join_data_per_code = pd.merge(recall_per_code,iou_per_code,on="code")
    join_data_per_code = join_data_per_code.set_index("code")
    join_data_per_code = join_data_per_code*100
    return join_data_per_code
    # print(f"Mean values across flood events: {join_data_per_code.mean(axis=0).to_dict()}")

In [34]:
import itertools

prod_dev = "0_DEV"
dl_name = "test"

experiments_stats = []

for e in experiments_dev:
    # Obtain band configuration and model
    #config = utils.read_json_from_gcp(f"gs://ml4cc_data_lake/{prod_dev}/2_Mart/2_MLModelMart/{e}/config.json")
    if e != 'MNDWI':
        config = load_json(rf"X:\home\kike\Projectes\ml4floods\2_MLModelMart\{e}/config.json")
        channels = config["model_params"]["hyperparameters"]["channel_configuration"]
        architecture = config["model_params"]["hyperparameters"]["model_type"]
    else:
        channels, architecture = 'MNDWI','MNDWI'
        
    
    for dl_name in ["val", "test"]:
        data = load_json(rf"X:\home\kike\Projectes\ml4floods\2_MLModelMart\{e}\{dl_name}.json")

        # compute metrics per flood
        metrics_per_code = get_metrics_percode(data)
        mean_metrics_code = metrics_per_code.mean(axis=0).to_dict()


        experiments_stats.append({
            "global_recall" : data["recall"]["water"],
            "global_iou" : data["iou"]["water"],
            "mean_recall_perflood" : mean_metrics_code["Recall_water"],
            "mean_iou_perflood" : mean_metrics_code["IoU_water"],
            "experiment": e,
            "channels": channels,
            "architecture": architecture,
            "subset": dl_name
        })

experiments_stats = pd.DataFrame(experiments_stats)
experiments_stats.to_clipboard(excel = True, sep = '\t')
experiments_stats

,global_recall,global_iou,mean_recall_perflood,mean_iou_perflood,experiment,channels,architecture,subset
0,0.956685,0.857792,82.975388,67.649748,WFV1_unet,all,unet,val
1,0.873288,0.778395,90.999290,81.467950,WFV1_unet,all,unet,test
2,0.968352,0.802382,89.681845,66.741732,WF1_unet_all,all,unet,val
3,0.905339,0.774883,93.286341,78.199284,WF1_unet_all,all,unet,test
4,0.963880,0.850193,87.409568,67.589705,WF1_unet_rgbiswir,bgriswir,unet,val
5,0.891573,0.724112,92.302272,77.662511,WF1_unet_rgbiswir,bgriswir,unet,test
6,0.959987,0.836739,87.336232,66.469892,WF1_unet_rgbi,bgri,unet,val
7,0.864218,0.696180,90.225599,71.054701,WF1_unet_rgbi,bgri,unet,test
8,0.957098,0.492340,84.429170,48.966239,WF1_hrnet_rgbbs32,rgb,hrnet_small,val
9,0.665169,0.533661,88.646990,63.465381,WF1_hrnet_rgbbs32,rgb,hrnet_small,test


In [38]:
#experiments_stats_subset = experiments_stats.groupby('subset')
experiments_stats_subset = experiments_stats.sort_values(by = ['subset','mean_iou_perflood'], ascending= False)
experiments_stats_subset


,global_recall,global_iou,mean_recall_perflood,mean_iou_perflood,experiment,channels,architecture,subset
0,0.956685,0.857792,82.975388,67.649748,WFV1_unet,all,unet,val
4,0.963880,0.850193,87.409568,67.589705,WF1_unet_rgbiswir,bgriswir,unet,val
2,0.968352,0.802382,89.681845,66.741732,WF1_unet_all,all,unet,val
10,0.968352,0.802382,89.681845,66.741732,WF1_unet_all,all,unet,val
6,0.959987,0.836739,87.336232,66.469892,WF1_unet_rgbi,bgri,unet,val
16,0.976877,0.818035,94.811966,63.915027,WF2_Unet_unused_from_checkpoint,all,unet,val
12,0.971566,0.766578,91.057589,61.302249,WF2_unet,all,unet,val
8,0.957098,0.492340,84.429170,48.966239,WF1_hrnet_rgbbs32,rgb,hrnet_small,val
14,0.978507,0.270768,89.978861,38.466622,WF2_hrnetsmall_rgb,rgb,hrnet_small,val
1,0.873288,0.778395,90.999290,81.467950,WFV1_unet,all,unet,test


In [13]:
metrics_per_code

,Recall_land,Recall_water,Recall_cloud,IoU_land,IoU_water,IoU_cloud
code,,,,,,
EMSR264,80.929519,86.440341,8.830603e+01,78.073280,45.861917,8.560192e+01
EMSR273,94.717759,97.095125,8.849557e-07,93.094014,89.215352,5.181347e-07
EMSR286,95.069232,99.650396,9.714461e+01,93.573385,51.884892,9.275057e+01
EMSR333,82.073973,99.198019,8.973068e+01,80.471368,42.307120,8.268879e+01
EMSR342,70.288910,93.591771,9.628920e+01,66.159082,72.803346,9.113866e+01
EMSR347,95.378611,93.430593,9.573262e+01,94.453020,66.346619,8.978987e+01
EMSR419,91.249249,97.368625,1.189729e+01,89.797281,85.109646,5.947796e+00
EMSR422,66.030463,99.385101,2.264631e+01,64.320950,90.984126,2.013575e+01
EMSR438,80.067964,99.342064,8.805451e+01,79.421044,81.687845,8.039811e+01
